# Powr of Delta Table History

In [0]:
%python
dbutils.widgets.text("the_table_name", "amitabh_arora_catalog.supply_chain.purchasing_suppliers", "Table Name")

## Change Volume
To monitor the number of rows affected by operations (e.g., INSERT, UPDATE, DELETE) on a Delta Table within a specific time period. This KPI helps track the table's activity level and identify whether there is a need to perform tuning. For example, OPTIMIZE operation to reduce the number of small files and improve query performance or understand the trends in number of updated/inserted/deleted records.

In [0]:
WITH history AS (
 -- Get the Delta Table history using DESC HISTORY
 DESCRIBE HISTORY amitabh_arora_catalog.supply_chain.purchasing_suppliers
),
delta_history AS (
 -- Flatten the history results and extract the necessary columns
 SELECT
   operation,
   operationMetrics["numOutputRows"] AS numOutputRows,  -- Number of rows affected
   operationMetrics["numFiles"] AS numFiles,
   operationMetrics["bytesAdded"] AS bytesAdded,             -- Number of files affected
   operationMetrics["bytesRemoved"] AS bytesRemoved,
   CAST(timestamp AS DATE) AS operation_date             -- Extract the date from the timestamp
 FROM
   history
)


-- Calculate the total rows affected, files impacted, and split by operation and date
SELECT
   operation_date,                   -- Group by date
   operation,                        -- Group by operation type (INSERT, UPDATE, DELETE, etc.)
   SUM(numOutputRows) AS total_rows_affected,  -- Total rows affected by operations per day
   SUM(numFiles) AS total_files_written,       -- Total files affected by operations per day
   SUM(bytesAdded) AS total_bytes_added,  -- Total bytes added by operations per day
   SUM(bytesRemoved) AS total_bytes_removed       -- Total bytes removed by operations per day
FROM
   delta_history
GROUP BY
   operation_date,                   -- Group by operation date
   operation                         -- Group by operation type (INSERT, UPDATE, DELETE)
ORDER BY
   operation_date DESC,              -- Sort by date (most recent first)
   operation; 

## Data Insertion/Update Speed (Execution Time)
The Data Insertion/Update Speed KPI measures the time it takes to perform a data modification operation on a Delta Table, such as an INSERT, UPDATE, or MERGE. This metric helps assess the performance of data modification operations in your Delta Lake pipeline, providing insights into how efficiently data is being ingested or updated.

Apart from total execution time, you can visualize scan time and re-write time as well.

In [0]:
WITH history AS (
 -- Get the Delta Table history using DESC HISTORY
 DESCRIBE HISTORY amitabh_arora_catalog.supply_chain.purchasing_suppliers
),
delta_history AS (
 -- Flatten the history results and extract the necessary columns
 SELECT
   operation,
   operationMetrics["executionTimeMs"] AS execution_time_ms,  -- Total execution time in milliseconds
   operationMetrics["scanTimeMs"] AS scan_time_ms,            -- Time taken for scanning data
   operationMetrics["rewriteTimeMs"] AS rewrite_time_ms,      -- Time taken for rewriting data
   CAST(timestamp AS DATE) AS operation_date  -- Extract the date from the timestamp
 FROM
   history
 WHERE
   operation IN ('WRITE', 'UPDATE', 'MERGE', 'WRITE')  -- Filter for data modification operations only
)


-- Calculate the execution time breakdown (Scan Time, Rewrite Time, and Other Time)
SELECT
   operation_date,                     -- Group by operation date
   operation,                          -- Group by operation type (WRITE, UPDATE, MERGE)
   AVG(scan_time_ms) AS avg_scan_time_ms,         -- Average scan time per operation
   AVG(rewrite_time_ms) AS avg_rewrite_time_ms,   -- Average rewrite time per operation
   AVG(execution_time_ms) AS avg_execution_time_ms,  -- Average total execution time
   MIN(scan_time_ms) AS min_scan_time_ms,  -- Minimum scan_time_ms
   MAX(scan_time_ms) AS max_scan_time_ms,   -- Maximum scan_time_ms
   MIN(rewrite_time_ms) AS min_rewrite_time_ms,  -- Minimum rewrite_time_ms
   MAX(rewrite_time_ms) AS max_rewrite_time_ms,   -- Maximum rewrite_time_ms
   MIN(execution_time_ms) AS min_execution_time_ms,  -- Minimum execution time
   MAX(execution_time_ms) AS max_execution_time_ms   -- Maximum execution time
FROM
   delta_history
GROUP BY
   operation_date,                     -- Group by operation date
   operation                           -- Group by operation type (WRITE, UPDATE, MERGE)
ORDER BY
   operation_date DESC,                -- Sort by date (most recent first)
   operation;                          -- Sort by operation type

## Most Active Users/Jobs Impacting Table
Tracking the most active users or jobs impacting table updates provides valuable insights into the behavior and health of your table, especially in cases where multiple processes and stakeholders are making changes. By monitoring who is making the most updates and understanding the volume of changes being made, you can proactively address issues, detect inefficiencies, and ensure smooth operation across all users and processes involved.

In [0]:
WITH history AS (
 -- Get the Delta Table history using DESC HISTORY
 DESCRIBE HISTORY amitabh_arora_catalog.supply_chain.purchasing_suppliers
),
delta_history AS (
 -- Flatten the history results and extract the necessary columns
 SELECT
   userName,                        -- The user making the modification
   job["jobid"] as jobid,                           -- The job executing the operation
   operation,                       -- The type of operation (WRITE, UPDATE, MERGE, etc.)
   operationMetrics.numOutputRows AS num_output_rows, -- Rows affected by the operation
   CAST(timestamp AS DATE) AS operation_date  -- Extract the date from the timestamp
 FROM
   history
 WHERE
   operation IN ('INSERT', 'UPDATE', 'MERGE', 'WRITE')  -- Filter for data modification operations only
)


-- Top Users (Only if jobId is NULL)
SELECT
   userName,                            -- Group by user
   COUNT(*) AS num_operations,          -- Count the number of operations performed by the user
   SUM(num_output_rows) AS total_rows_affected  -- Sum of rows affected by the user
FROM
   delta_history
WHERE
   jobid IS NULL  -- Only consider users with NULL jobId
GROUP BY
   userName
ORDER BY
   num_operations DESC,                 -- Sort by number of operations (most active first)
   total_rows_affected DESC             -- Sort by the number of rows affected
LIMIT 10   -- Limit the results to top 10 users
;


-- Top Jobs (Only if jobId is NOT NULL)
SELECT
   jobId,                               -- Group by job
   COUNT(*) AS num_operations,          -- Count the number of operations performed by the job
   SUM(num_output_rows) AS total_rows_affected  -- Sum of rows affected by the job
FROM
   delta_history
WHERE
   jobId IS NOT NULL  -- Only consider jobs with non-null jobId
GROUP BY
   jobId
ORDER BY
   num_operations DESC,                 -- Sort by number of operations (most active first)
   total_rows_affected DESC             -- Sort by the number of rows affected
LIMIT 10   -- Limit the results to top 10 jobs